In [2]:
pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.3 MB/s eta 0:00:00


In [3]:
pip install networkx

In [4]:
pip install geopandas shapely matplotlib

In [1]:
from geopy.distance import geodesic
import osmnx as ox
import networkx as nx

def calculate_distance(coord1, coord2, use_road_network=True):
    """
    Calculate the distance between two GPS coordinates.

    Args:
        coord1 (tuple): The first coordinate as (latitude, longitude).
        coord2 (tuple): The second coordinate as (latitude, longitude).
        use_road_network (bool): If True, calculates distance using road network.
                                 If False, calculates straight-line geodesic distance.

    Returns:
        float: Distance in meters.
    """
    if not use_road_network:
        # Calculate geodesic distance
        distance_meters = geodesic(coord1, coord2).meters
        return distance_meters
    else:
        # Download road network for the area
        G = ox.graph_from_point(coord1, dist=5000, network_type='drive')

        # Find the nearest nodes in the road network
        node1 = ox.distance.nearest_nodes(G, coord1[1], coord1[0])
        node2 = ox.distance.nearest_nodes(G, coord2[1], coord2[0])

        # Calculate the shortest path distance using the road network
        try:
            distance_meters = nx.shortest_path_length(G, node1, node2, weight='length')
        except nx.NetworkXNoPath:
            distance_meters = None  # No path exists between the two points

        return distance_meters

# Example Usage
coord1 = (54.78769, 9.4651324)  # Bridge location.
coord2 = (54.78795, 9.3374747)   # Seimens provided location near/faraway the bridge.

# Geodesic distance (straight-line)
geo_distance = calculate_distance(coord1, coord2, use_road_network=False)
print(f"Geodesic distance: {geo_distance:.2f} meters")

# Road network distance
road_distance = calculate_distance(coord1, coord2, use_road_network=True)
if road_distance is not None:
    print(f"Road network distance: {road_distance:.2f} meters")
else:
    print("No road network path found between the points.")

Geodesic distance: 8212.45 meters
Road network distance: 7492.77 meters
